Talvez eu possa fazer com que as funções que exportam as tabelas façam isso mais detalhe. Com isso, posso colocar na função de impressão os filtros necessários para que haja seleção do tipo de nível granular que deve aparecer no gráfico.

In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
pd.options.display.float_format = '{:,.2f}'.format

from itertools import product

import matplotlib.pyplot as plt

import plotly.express as px

import bamboolib as bam
bam.enable()
#bam.disable()

import plotly.express as px

import app_rais.potec as potec

Success: the bamboolib extension was enabled successfully. You can disable it via 'bam.disable()'. You will now see a magic bamboolib button when you display your dataframes, for example via 'df'


In [ ]:
def load_base(ufs = ('pr', 'rs', 'pe'), year = 2015, data_format='wide', filter_metarea=True):
    """
    For selected Brazilian Federation Unities (ufs) and year, loads pretreated Rais data into a Pandas DataFrame and treat it so 
    it is suited to analysis of formally employed personnel with respect to metrics of: scientific and technical (S&T)
    personnel (POTEC), organization legal status, economic sectors, categories of services according to intensity of applied 
    knowledge, industry technological levels; and schoolling.
    
    If filter_metarea is set to True, it will load data reffering just to the metropolitan areas of the selected Federation Unities,
    leaving out data of remaining municipalities.
    
    """
    
    multi_index=True
    dict_df = dict()
    dict_df_ufs = dict()
    dict_rms = {'PR': 'Curitiba', 'RS':'Porto Alegre', 'PE': 'Recife'}

    for uf in ufs:

        df = pd.read_csv(
            f'app_rais/data/rais_treated/{year}/{uf.upper()}{year}.zip'
            , compression='zip'
            , header=0
            , sep=';'
            , decimal=','
            , dtype=dic_dtype
            , usecols=[
                'territorio_tese'
#                , 'Município'                
#                , 'CBO Ocupação 2002'
#                , 'CNAE 2.0 Classe'
#                , 'Faixa Tempo Emprego'
                , 'Escolaridade após 2005'
                , 'Natureza Jurídica'
                , 'Tamanho Estabelecimento'
                , 'knowledge_services'
                , 'technology_industries'
                , 'potec'
            ]
        )
        
        
#        df['UF'] = uf.upper()
        
        if filter_metarea == True:
            filter_rm_territory = df.territorio_tese != df.territorio_tese.cat.categories[1]
            filter_potec = df.potec != df.potec.cat.categories[-1]
            df = df.loc[filter_rm_territory, :].groupby(by=['territorio_tese', 'Tamanho Estabelecimento', 'Natureza Jurídica', 'knowledge_services', 'technology_industries', 'Escolaridade após 2005','potec'], observed=True).size().reset_index()
        
        elif filter_metarea == False:
            df = df.groupby(by=['territorio_tese', 'Tamanho Estabelecimento', 'Natureza Jurídica', 'knowledge_services', 'technology_industries', 'Escolaridade após 2005','potec'], observed=True).size().reset_index()
        
        
        dict_df[uf] = df

        
    df = pd.concat(dict_df.values(),  axis=0)
    df.rename(columns={0: 'Pessoal'}, inplace=True)
    
    
    df.territorio_tese = df.territorio_tese.astype('category')
    df.rename(columns={'territorio_tese':'Espaço Metropolitano'}, inplace=True)
    df['Espaço Metropolitano'] = df['Espaço Metropolitano'].str.slice_replace(start=0, stop=24).astype('category')
    df['Tamanho Estabelecimento'] = df['Tamanho Estabelecimento'].map(dict_porte).astype(type_porte)
    df['Natureza Jurídica Grupo'] = df['Natureza Jurídica'].map(dict_nat_jur).astype('category')
    df['Natureza Jurídica'] = df['Natureza Jurídica'].map(dict_nat_jur_detail).astype('category')
    
    df['Sectors'] = df.knowledge_services.map({'Without Classification':'Without Classification'}).fillna('Services')
    df['Sectors'] = ['Services' if x[1]['Sectors'] == 'Services' else 'Industry' if x[1]['technology_industries'] != 'Without Classification' and x[1]['Sectors'] != 'Services'  else 'Without Classification' for x in df.iterrows()]
    df['Sectors'] = df['Sectors'].astype('category')
    
    
    df['Escolaridade1'] = df['Escolaridade após 2005'].map(dict_escolaridade)
    df['Escolaridade2'] = df['Escolaridade após 2005'].map(dict_escolaridade1)
    
    df['Potec Grupo'] = df['potec'].map(dict_potec).astype('category')
    
    df = df[[
        'Espaço Metropolitano'
        , 'Sectors'
        , 'Tamanho Estabelecimento'
        , 'Natureza Jurídica Grupo'
        , 'Natureza Jurídica'
        , 'knowledge_services'
        , 'technology_industries'
    #    , 'Escolaridade após 2005',
        , 'Potec Grupo'
        , 'potec'
        , 'Escolaridade1'
        , 'Escolaridade2'
        , 'Pessoal'
    ]]
    
        
    return df

In [2]:
list_column_keep_2010 = [
    'CBO Ocupação 2002'
    , 'CNAE 2.0 Classe'
    , 'CNAE 95 Classe'
    , 'Vínculo Ativo 31/12'
    , 'Faixa Etária'
    , 'Faixa Remun Dezem (SM)'
    , 'Faixa Remun Média (SM)'
    , 'Faixa Tempo Emprego'
    , 'Escolaridade após 2005'
    , 'Mun Trab'
    , 'Município'
    , 'Nacionalidade'
    , 'Natureza Jurídica'
    , 'Vl Remun Dezembro Nom'
    , 'Vl Remun Dezembro (SM)'
    , 'Vl Remun Média Nom'
    , 'Vl Remun Média (SM)'
    , 'CNAE 2.0 Subclasse'
    , 'Tamanho Estabelecimento'
    , 'Tipo Estab'
    , 'Tipo Estab.1'
    , 'Tipo Vínculo'
]

type_cat_tam_estabelecimento = CategoricalDtype(categories=[ -1, 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], ordered=True)
type_cat_escolaridade = CategoricalDtype(categories=[ -1, 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], ordered=True)
type_cat_faixa_etaria = CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08'], ordered=True)
type_cat_faixa_remu_media = CategoricalDtype(
    categories=[
        '00'
        ,'01'
        , '02'
        , '03'
        , '04'
        , '05'
        , '06'
        , '07'
        , '08'
        , '09'
        , '10'
        , '11'
    ]
    , ordered=True
)

type_cat_faixa_remu_media_dezembro = CategoricalDtype(
    categories=[
        '00'
        , '01'
        , '02'
        , '03'
        , '04'
        , '05'
        , '06'
        , '07'
        , '08'
        , '09'
        , '10'
        , '11'
        , '12'
    ]
    , ordered=True
)

type_cat_ind_tec = CategoricalDtype(
    categories=[
        'High-technology'
        , 'Medium-high-technology'
        , 'Medium-low-technology'
        , 'Low-technology'
        , 'Without Classification'
    ]
    , ordered=True
)


type_cat_know_services = CategoricalDtype(
    categories=[
        'High-tech knowledge-intensive services'
        , 'Knowledge-intensive financial services'
        , 'Knowledge-intensive market services'
        , 'Other knowledge-intensive services'
        , 'Less knowledge-intensive market services'
        , 'Other less knowledge-intensive services'
        , 'Without Classification'
    ]
    , ordered=True
)

type_cat_potec_grupo = CategoricalDtype(
    categories= ['Profissionais do Ensino', 'Engenheiros', 'Profissionais científicos', 'Pesquisadores', 'Diretores e Gerentes de P&D']    
        , ordered=True
)

type_cat_potec = CategoricalDtype(
    categories= ['Diretores e Gerentes de P&D'
                 ,'Engenheiros Mecatrônicos'
                 ,'Engenheiros Civis etc'
                 ,'Engenheiros agrônomos, de alimentos, florestais e de pesca'
                 ,'Pesquisadores'
                 ,'Biotecnologistas, geneticistas, pesquisadores em metrologia e especialistas em calibrações metereológicas'
                 ,'Matemáticos, estatísticos e afins'
                 ,'Profissionais da Informática'
                 ,'Físicos, químicos e afins'
                 ,'Biólogos e biomédicos'
                 ,'Professores de ensino Profissional'
                 ,'Professores de ensino superior'
                 ,'Demais Ocupações'
                ]    
        , ordered=True
)


dic_dtype = {
    'CBO Ocupação 2002' : 'category'
    ,'CNAE 2.0 Classe' : 'category'
    ,'CNAE 95 Classe' : 'category'
    ,'Faixa Etária' : type_cat_faixa_etaria
    ,'Faixa Remun Dezem (SM)' : type_cat_faixa_remu_media_dezembro
    ,'Faixa Remun Média (SM)' : type_cat_faixa_remu_media
    ,'Faixa Tempo Emprego' : 'category'
    ,'Escolaridade após 2005' : type_cat_escolaridade
    ,'Mun Trab' : 'category'
    ,'Município' : 'category'
    ,'Nacionalidade' : 'category'
    ,'Natureza Jurídica' : 'category'
    ,'CNAE 2.0 Subclasse' : 'category'
    ,'Tamanho Estabelecimento' : type_cat_tam_estabelecimento
    ,'Tipo Estab' : 'category'
    ,'Tipo Estab.1' : 'category'
    ,'Tipo Vínculo' : 'category'
    , 'Vl Remun Dezembro Nom' : np.float64
    , 'Vl Remun Média Nom' : np.float32
    , 'Vl Remun Dezembro (SM)' : np.float64
    , 'Vl Remun Média (SM)' : np.float64
    , 'Tempo Emprego' : np.float64
    , 'territorio_tese':'category'
    , 'arranjo':'category'
    , 'knowledge_services':'category'
    , 'technology_industries':type_cat_ind_tec
    , 'potec':type_cat_potec
}

dict_porte = {
    -1:'Ignorado'
    , 1:'Até 49'
    , 2:'Até 49'
    , 3:'Até 49'
    ,4:'Até 49'
    ,5:'Até 49'
    ,6:'de 50 a 99'
    ,7:'de 100 a 249'
    ,8:'de 250 a 499'
    ,9:'de 500 a 999'
    ,10:'a partir de 1000'
}


type_porte = CategoricalDtype(
    categories=[
        'Ignorado'
        ,'Até 49'
        ,'de 50 a 99'
        ,'de 100 a 249'
        ,'de 250 a 499'
        ,'de 500 a 999'
        ,'a partir de 1000'
    ]
    , ordered=True
)

dict_potec = {'Diretores e Gerentes de P&D':'Diretores e Gerentes de P&D'
              ,'Engenheiros Mecatrônicos':'Engenheiros'
              ,'Engenheiros Civis etc':'Engenheiros'
              ,'Engenheiros agrônomos, de alimentos, florestais e de pesca':'Engenheiros' 
              ,'Pesquisadores':'Pesquisadores'
              ,'Biotecnologistas, geneticistas, pesquisadores em metrologia e especialistas em calibrações metereológicas':'Profissionais científicos'
              ,'Matemáticos, estatísticos e afins':'Profissionais científicos'
              ,'Profissionais da Informática':'Profissionais científicos'
              ,'Físicos, químicos e afins':'Profissionais científicos'
              ,'Biólogos e biomédicos':'Profissionais científicos'
              ,'Professores de ensino Profissional':'Profissionais do Ensino'
              ,'Professores de ensino superior':'Profissionais do Ensino'
              ,'Demais Ocupações':'Demais Ocupações'
             }


dict_nat_jur={
    '1015':'Administração Pública'
    ,'1023':'Administração Pública'
    ,'1031':'Administração Pública'
    ,'1040':'Administração Pública'
    ,'1058':'Administração Pública'
    ,'1066':'Administração Pública'
    ,'1074':'Administração Pública'
    ,'1082':'Administração Pública'
    ,'1104':'Administração Pública'
    ,'1112':'Administração Pública'
    ,'1120':'Administração Pública'
    ,'1139':'Administração Pública'
    ,'1147':'Administração Pública'
    ,'1155':'Administração Pública'
    ,'1163':'Administração Pública'
    ,'1171':'Administração Pública'
    ,'1180':'Administração Pública'
    ,'1198':'Administração Pública'
    ,'1201':'Administração Pública'
    ,'1210':'Administração Pública'
    ,'1228':'Administração Pública'
    ,'1236':'Administração Pública'
    ,'1244':'Administração Pública'
    ,'1252':'Administração Pública'
    ,'1260':'Administração Pública'
    ,'1279':'Administração Pública'
    ,'2011':'Entidades Empresariais'
    ,'2038':'Entidades Empresariais'
    ,'2046':'Entidades Empresariais'
    ,'2054':'Entidades Empresariais'
    ,'2062':'Entidades Empresariais'
    ,'2070':'Entidades Empresariais'
    ,'2089':'Entidades Empresariais'
    ,'2097':'Entidades Empresariais'
    ,'2127':'Entidades Empresariais'
    ,'2135':'Entidades Empresariais'
    ,'2143':'Entidades Empresariais'
    ,'2151':'Entidades Empresariais'
    ,'2160':'Entidades Empresariais'
    ,'2178':'Entidades Empresariais'
    ,'2194':'Entidades Empresariais'
    ,'2216':'Entidades Empresariais'
    ,'2224':'Entidades Empresariais'
    ,'2232':'Entidades Empresariais'
    ,'2240':'Entidades Empresariais'
    ,'2259':'Entidades Empresariais'
    ,'2267':'Entidades Empresariais'
    ,'2275':'Entidades Empresariais'
    ,'2283':'Entidades Empresariais'
    ,'2291':'Entidades Empresariais'
    ,'2305':'Entidades Empresariais'
    ,'2313':'Entidades Empresariais'
    ,'3034':'Entidades sem Fins Lucrativos'
    ,'3069':'Entidades sem Fins Lucrativos'
    ,'3077':'Entidades sem Fins Lucrativos'
    ,'3085':'Entidades sem Fins Lucrativos'
    ,'3107':'Entidades sem Fins Lucrativos'
    ,'3115':'Entidades sem Fins Lucrativos'
    ,'3131':'Entidades sem Fins Lucrativos'
    ,'3204':'Entidades sem Fins Lucrativos'
    ,'3212':'Entidades sem Fins Lucrativos'
    ,'3220':'Entidades sem Fins Lucrativos'
    ,'3239':'Entidades sem Fins Lucrativos'
    ,'3247':'Entidades sem Fins Lucrativos'
    ,'3255':'Entidades sem Fins Lucrativos'
    ,'3263':'Entidades sem Fins Lucrativos'
    ,'3271':'Entidades sem Fins Lucrativos'
    ,'3280':'Entidades sem Fins Lucrativos'
    ,'3298':'Entidades sem Fins Lucrativos'
    ,'3301':'Entidades sem Fins Lucrativos'
    ,'3999':'Entidades sem Fins Lucrativos'
    ,'4014':'Pessoas Físicas'
    ,'4022':'Pessoas Físicas'
    ,'4081':'Pessoas Físicas'
    ,'4090':'Pessoas Físicas'
    ,'4111':'Pessoas Físicas'
    ,'4120':'Pessoas Físicas'
    ,'5010':'Organizações Internacionais'
    ,'5029':'Organizações Internacionais'
    ,'5037':'Organizações Internacionais'
    ,'-1':'Ignorado'
}


dict_nat_jur_detail = {
    '1015':'POD EXEC FE'
    ,'1023':'POD EXEC ES'
    ,'1031':'POD EXEC MU'
    ,'1040':'POD LEG FED'
    ,'1058':'POD LEG EST'
    ,'1066':'POD LEG MUN'
    ,'1074':'POD JUD FED'
    ,'1082':'POD JUD EST'
    ,'1104':'AUTARQ FED'
    ,'1112':'AUTARQ EST'
    ,'1120':'AUTARQ MUN'
    ,'1139':'FUNDAC FED'
    ,'1147':'FUNDAC EST'
    ,'1155':'FUNDAC MUN'
    ,'1163':'ORG AUT FED'
    ,'1171':'ORG AUT EST'
    ,'1180':'ORG AUT MUN'
    ,'1198':'COM POLINAC'
    ,'1201':'FUNDO PUBLIC'
    ,'1210':'ASSOC PUBLIC'
    ,'1228':'CONS PUB D PRIV'
    ,'1236':'UF'
    ,'1244':'MUN'
    ,'1252':'FUN PUB PRIV FED'
    ,'1260':'FUN PUB PRIV UF'
    ,'1279':'FUN PUB PRIV MUN'
    ,'2011':'EMP PUB'
    ,'2038':'SOC MISTA'
    ,'2046':'SA ABERTA'
    ,'2054':'SA FECH'
    ,'2062':'SOC QT LTDA'
    ,'2070':'SOC COLETV'
    ,'2089':'SOC COMD SM'
    ,'2097':'SOC COMD AC'
    ,'2127':'SOC CTA PAR'
    ,'2135':'FRM MER IND'
    ,'2143':'COOPERATIVA'
    ,'2151':'CONS EMPRES'
    ,'2160':'GRUP SOC'
    ,'2178':'FIL EMP EXT'
    ,'2194':'FIL ARG-BRA'
    ,'2216':'EMP DOM EXT'
    ,'2224':'FUN INVEST'
    ,'2232':'SOC SIMP PUR'
    ,'2240':'SOC SIMP LTD'
    ,'2259':'SOC SIMP COL'
    ,'2267':'SOC SIMP COM'
    ,'2275':'EMPR BINAC'
    ,'2283':'CONS EMPREG'
    ,'2291':'CONS SIMPLES'
    ,'2305':'EMP IND LTDA'
    ,'2313':'EMP IND SIMPLES'
    ,'3034':'CARTORIO'
    ,'3069':'OUT FUND PR'
    ,'3077':'SERV SOC AU'
    ,'3085':'CONDOMIN'
    ,'3107':'COM CONC'
    ,'3115':'ENT MED ARB'
    ,'3131':'ENT SOCIAL07'
    ,'3204':'FIL FUN EXT'
    ,'3212':'FUN DOM EXT'
    ,'3220':'ORG RELIG'
    ,'3239':'COMUN INDIG'
    ,'3247':'FUNDO PRIVAD'
    ,'3255':'PARTIDO'
    ,'3263':'PARTIDO'
    ,'3271':'PARTIDO'
    ,'3280':'PARTIDO'
    ,'3298':'FRENTE PLEB'
    ,'3301':'OS'
    ,'3999':'OUTR ORG'
    ,'4014':'EMP IND IMO'
    ,'4022':'SEG ESPEC'
    ,'4081':'CONTR IND07'
    ,'4090':'CAN CARG POL'
    ,'4111':'LEILOEIRO'
    ,'4120':'PROD RURAL'
    ,'5010':'ORG INTERNAC'
    ,'5029':'REPR DIPL ES'
    ,'5037':'OUT INST EXT'
    ,'-1':'INGNORADO'  
}


dict_escolaridade = {
    -1:'Não-Superior'
    ,1:'Não-Superior'
    ,2:'Não-Superior'
    ,3:'Não-Superior'
    ,4:'Não-Superior'
    ,5:'Não-Superior'
    ,6:'Não-Superior'
    ,7:'Não-Superior'
    ,8:'Não-Superior'
    ,9:'Superior'
    ,10:'Mestrado'
    ,11:'Doutorado'
}

dict_escolaridade1 = {
    -1:'Inferior ao Ensino Médio Completo'
    ,1:'Inferior ao Ensino Médio Completo'
    ,2:'Inferior ao Ensino Médio Completo'
    ,3:'Inferior ao Ensino Médio Completo'
    ,4:'Inferior ao Ensino Médio Completo'
    ,5:'Inferior ao Ensino Médio Completo'
    ,6:'Inferior ao Ensino Médio Completo'
    ,7:'Ensino Médio ao Superior Incompleto'
    ,8:'Ensino Médio ao Superior Incompleto'
    ,9:'Superior'
    ,10:'Mestrado'
    ,11:'Doutorado'
}

In [3]:
def load_base(ufs = ('pr', 'rs', 'pe'), year = 2015, data_format='wide', filter_metarea=True):
    """
    For selected Brazilian Federation Unities (ufs) and year, loads pretreated Rais data into a Pandas DataFrame and treat it so 
    it is suited to analysis of formally employed personnel with respect to metrics of: scientific and technical (S&T)
    personnel (POTEC), organization legal status, economic sectors, categories of services according to intensity of applied 
    knowledge, industry technological levels; and schoolling.
    
    If filter_metare is set to True, it will load data reffering just to the metropolitan areas of the selected Federation Unities,
    leaving out data of remaining municipalities.
    
    """
    
    multi_index=True
    dict_df = dict()
    dict_df_ufs = dict()
    dict_rms = {'PR': 'Curitiba', 'RS':'Porto Alegre', 'PE': 'Recife'}

    for uf in ufs:

        df = pd.read_csv(
            f'app_rais/data/rais_treated/{year}/{uf.upper()}{year}.zip'
            , compression='zip'
            , header=0
            , sep=';'
            , decimal=','
            , dtype=dic_dtype
            , usecols=[
                'territorio_tese'
#                , 'Município'                
#                , 'CBO Ocupação 2002'
#                , 'CNAE 2.0 Classe'
#                , 'Faixa Tempo Emprego'
                , 'Escolaridade após 2005'
                , 'Natureza Jurídica'
                , 'Tamanho Estabelecimento'
                , 'knowledge_services'
                , 'technology_industries'
                , 'potec'
            ]
        )
        
        
#        df['UF'] = uf.upper()
        
        if filter_metarea == True:
            filter_rm_territory = df.territorio_tese != df.territorio_tese.cat.categories[1]
            filter_potec = df.potec != df.potec.cat.categories[-1]
            df = df.loc[filter_rm_territory, :].groupby(by=['territorio_tese', 'Tamanho Estabelecimento', 'Natureza Jurídica', 'knowledge_services', 'technology_industries', 'Escolaridade após 2005','potec'], observed=True).size().reset_index()
        
        elif filter_metarea == False:
            df = df.groupby(by=['territorio_tese', 'Tamanho Estabelecimento', 'Natureza Jurídica', 'knowledge_services', 'technology_industries', 'Escolaridade após 2005','potec'], observed=True).size().reset_index()
        
        
        dict_df[uf] = df

        
    df = pd.concat(dict_df.values(),  axis=0)
    df.rename(columns={0: 'Pessoal'}, inplace=True)
    
    
    df.territorio_tese = df.territorio_tese.astype('category')
    df.rename(columns={'territorio_tese':'Espaço Metropolitano'}, inplace=True)
    df['Espaço Metropolitano'] = df['Espaço Metropolitano'].str.slice_replace(start=0, stop=24).astype('category')
    df['Tamanho Estabelecimento'] = df['Tamanho Estabelecimento'].map(dict_porte).astype(type_porte)
    df['Natureza Jurídica Grupo'] = df['Natureza Jurídica'].map(dict_nat_jur).astype('category')
    df['Natureza Jurídica'] = df['Natureza Jurídica'].map(dict_nat_jur_detail).astype('category')
    
    df['Sectors'] = df.knowledge_services.map({'Without Classification':'Without Classification'}).fillna('Services')
    df['Sectors'] = ['Services' if x[1]['Sectors'] == 'Services' else 'Industry' if x[1]['technology_industries'] != 'Without Classification' and x[1]['Sectors'] != 'Services'  else 'Without Classification' for x in df.iterrows()]
    df['Sectors'] = df['Sectors'].astype('category')
    
    
    df['Escolaridade1'] = df['Escolaridade após 2005'].map(dict_escolaridade)
    df['Escolaridade2'] = df['Escolaridade após 2005'].map(dict_escolaridade1)
    
    df['Potec Grupo'] = df['potec'].map(dict_potec).astype('category')
    
    df = df[[
        'Espaço Metropolitano'
        , 'Sectors'
        , 'Tamanho Estabelecimento'
        , 'Natureza Jurídica Grupo'
        , 'Natureza Jurídica'
        , 'knowledge_services'
        , 'technology_industries'
    #    , 'Escolaridade após 2005',
        , 'Potec Grupo'
        , 'potec'
        , 'Escolaridade1'
        , 'Escolaridade2'
        , 'Pessoal'
    ]]
    
        
    return df

In [10]:
%%time
df_2018_sul = pd.read_table(filepath_or_buffer='app_rais/data/rais_original/2018/RAIS_VINC_PUB_SUL.txt', encoding='Latin1', sep=';', decimal=',')

Wall time: 2min 29s


In [11]:
df_2018_sul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12065026 entries, 0 to 12065025
Data columns (total 62 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Bairros SP               object 
 1   Bairros Fortaleza        object 
 2   Bairros RJ               object 
 3   Causa Afastamento 1      int64  
 4   Causa Afastamento 2      int64  
 5   Causa Afastamento 3      int64  
 6   Motivo Desligamento      int64  
 7   CBO Ocupação 2002        object 
 8   CNAE 2.0 Classe          int64  
 9   CNAE 95 Classe           int64  
 10  Distritos SP             object 
 11  Vínculo Ativo 31/12      int64  
 12  Faixa Etária             int64  
 13  Faixa Hora Contrat       int64  
 14  Faixa Remun Dezem (SM)   int64  
 15  Faixa Remun Média (SM)   int64  
 16  Faixa Tempo Emprego      int64  
 17  Escolaridade após 2005   int64  
 18  Qtd Hora Contr           int64  
 19  Idade                    int64  
 20  Ind CEI Vinculado        int64  
 21  Ind Si

In [ ]:
list_column_keep_2010 = [
    'CBO Ocupação 2002'
    , 'CNAE 2.0 Classe'
    , 'CNAE 95 Classe'
    , 'Faixa Etária'
    , 'Faixa Remun Dezem (SM)'
    , 'Faixa Remun Média (SM)'
    , 'Faixa Tempo Emprego'
    , 'Escolaridade após 2005'
    , 'Mun Trab'
    , 'Município'
    , 'Nacionalidade'
    , 'Natureza Jurídica'
    , 'Vl Remun Dezembro Nom'
    , 'Vl Remun Dezembro (SM)'
    , 'Vl Remun Média Nom'
    , 'Vl Remun Média (SM)'
    , 'CNAE 2.0 Subclasse'
    , 'Tamanho Estabelecimento'
    , 'Tipo Estab'
    , 'Tipo Estab.1'
    , 'Tipo Vínculo'
]

type_cat_tam_estabelecimento = CategoricalDtype(categories=[ -1, 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], ordered=True)
type_cat_escolaridade = CategoricalDtype(categories=[ -1, 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], ordered=True)
type_cat_faixa_etaria = CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08'], ordered=True)
type_cat_faixa_remu_media = CategoricalDtype(
    categories=[
        '00'
        ,'01'
        , '02'
        , '03'
        , '04'
        , '05'
        , '06'
        , '07'
        , '08'
        , '09'
        , '10'
        , '11'
    ]
    , ordered=True
)

type_cat_faixa_remu_media_dezembro = CategoricalDtype(
    categories=[
        '00'
        , '01'
        , '02'
        , '03'
        , '04'
        , '05'
        , '06'
        , '07'
        , '08'
        , '09'
        , '10'
        , '11'
        , '12'
    ]
    , ordered=True
)

type_cat_ind_tec = CategoricalDtype(
    categories=[
        'High-technology'
        , 'Medium-high-technology'
        , 'Medium-low-technology'
        , 'Low-technology'
        , 'Without Classification'
    ]
    , ordered=True
)


type_cat_know_services = CategoricalDtype(
    categories=[
        'High-tech knowledge-intensive services'
        , 'Knowledge-intensive financial services'
        , 'Knowledge-intensive market services'
        , 'Other knowledge-intensive services'
        , 'Less knowledge-intensive market services'
        , 'Other less knowledge-intensive services'
        , 'Without Classification'
    ]
    , ordered=True
)

type_cat_potec_grupo = CategoricalDtype(
    categories= ['Profissionais do Ensino', 'Engenheiros', 'Profissionais científicos', 'Pesquisadores', 'Diretores e Gerentes de P&D']    
        , ordered=True
)

type_cat_potec = CategoricalDtype(
    categories= ['Diretores e Gerentes de P&D'
                 ,'Engenheiros Mecatrônicos'
                 ,'Engenheiros Civis etc'
                 ,'Engenheiros agrônomos, de alimentos, florestais e de pesca'
                 ,'Pesquisadores'
                 ,'Biotecnologistas, geneticistas, pesquisadores em metrologia e especialistas em calibrações metereológicas'
                 ,'Matemáticos, estatísticos e afins'
                 ,'Profissionais da Informática'
                 ,'Físicos, químicos e afins'
                 ,'Biólogos e biomédicos'
                 ,'Professores de ensino Profissional'
                 ,'Professores de ensino superior'
                 ,'Demais Ocupações'
                ]    
        , ordered=True
)


dic_dtype = {
    'CBO Ocupação 2002' : 'category'
    ,'CNAE 2.0 Classe' : 'category'
    ,'CNAE 95 Classe' : 'category'
    ,'Faixa Etária' : type_cat_faixa_etaria
    ,'Faixa Remun Dezem (SM)' : type_cat_faixa_remu_media_dezembro
    ,'Faixa Remun Média (SM)' : type_cat_faixa_remu_media
    ,'Faixa Tempo Emprego' : 'category'
    ,'Escolaridade após 2005' : type_cat_escolaridade
    ,'Mun Trab' : 'category'
    ,'Município' : 'category'
    ,'Nacionalidade' : 'category'
    ,'Natureza Jurídica' : 'category'
    ,'CNAE 2.0 Subclasse' : 'category'
    ,'Tamanho Estabelecimento' : type_cat_tam_estabelecimento
    ,'Tipo Estab' : 'category'
    ,'Tipo Estab.1' : 'category'
    ,'Tipo Vínculo' : 'category'
    , 'Vl Remun Dezembro Nom' : np.float64
    , 'Vl Remun Média Nom' : np.float32
    , 'Vl Remun Dezembro (SM)' : np.float64
    , 'Vl Remun Média (SM)' : np.float64
    , 'Tempo Emprego' : np.float64
    , 'territorio_tese':'category'
    , 'arranjo':'category'
    , 'knowledge_services':'category'
    , 'technology_industries':type_cat_ind_tec
    , 'potec':type_cat_potec
}

dict_porte = {
    -1:'Ignorado'
    , 1:'Até 49'
    , 2:'Até 49'
    , 3:'Até 49'
    ,4:'Até 49'
    ,5:'Até 49'
    ,6:'de 50 a 99'
    ,7:'de 100 a 249'
    ,8:'de 250 a 499'
    ,9:'de 500 a 999'
    ,10:'a partir de 1000'
}


type_porte = CategoricalDtype(
    categories=[
        'Ignorado'
        ,'Até 49'
        ,'de 50 a 99'
        ,'de 100 a 249'
        ,'de 250 a 499'
        ,'de 500 a 999'
        ,'a partir de 1000'
    ]
    , ordered=True
)

dict_potec = {'Diretores e Gerentes de P&D':'Diretores e Gerentes de P&D'
              ,'Engenheiros Mecatrônicos':'Engenheiros'
              ,'Engenheiros Civis etc':'Engenheiros'
              ,'Engenheiros agrônomos, de alimentos, florestais e de pesca':'Engenheiros' 
              ,'Pesquisadores':'Pesquisadores'
              ,'Biotecnologistas, geneticistas, pesquisadores em metrologia e especialistas em calibrações metereológicas':'Profissionais científicos'
              ,'Matemáticos, estatísticos e afins':'Profissionais científicos'
              ,'Profissionais da Informática':'Profissionais científicos'
              ,'Físicos, químicos e afins':'Profissionais científicos'
              ,'Biólogos e biomédicos':'Profissionais científicos'
              ,'Professores de ensino Profissional':'Profissionais do Ensino'
              ,'Professores de ensino superior':'Profissionais do Ensino'
              ,'Demais Ocupações':'Demais Ocupações'
             }

['3050', '2076', '3042', '5002', '2208', '3123', '3093', '2119']


dict_nat_jur={
    '1015':'Administração Pública'
    ,'1023':'Administração Pública'
    ,'1031':'Administração Pública'
    ,'1040':'Administração Pública'
    ,'1058':'Administração Pública'
    ,'1066':'Administração Pública'
    ,'1074':'Administração Pública'
    ,'1082':'Administração Pública'
    ,'1104':'Administração Pública'
    ,'1112':'Administração Pública'
    ,'1120':'Administração Pública'
    ,'1139':'Administração Pública'
    ,'1147':'Administração Pública'
    ,'1155':'Administração Pública'
    ,'1163':'Administração Pública'
    ,'1171':'Administração Pública'
    ,'1180':'Administração Pública'
    ,'1198':'Administração Pública'
    ,'1201':'Administração Pública'
    ,'1210':'Administração Pública'
    ,'1228':'Administração Pública'
    ,'1236':'Administração Pública'
    ,'1244':'Administração Pública'
    ,'1252':'Administração Pública'
    ,'1260':'Administração Pública'
    ,'1279':'Administração Pública'
    ,'2011':'Entidades Empresariais'
    ,'2038':'Entidades Empresariais'
    ,'2046':'Entidades Empresariais'
    ,'2054':'Entidades Empresariais'
    ,'2062':'Entidades Empresariais'
    ,'2070':'Entidades Empresariais'
    ,'2089':'Entidades Empresariais'
    ,'2097':'Entidades Empresariais'
    ,'2127':'Entidades Empresariais'
    ,'2135':'Entidades Empresariais'
    ,'2143':'Entidades Empresariais'
    ,'2151':'Entidades Empresariais'
    ,'2160':'Entidades Empresariais'
    ,'2178':'Entidades Empresariais'
    ,'2194':'Entidades Empresariais'
    ,'2216':'Entidades Empresariais'
    ,'2224':'Entidades Empresariais'
    ,'2232':'Entidades Empresariais'
    ,'2240':'Entidades Empresariais'
    ,'2259':'Entidades Empresariais'
    ,'2267':'Entidades Empresariais'
    ,'2275':'Entidades Empresariais'
    ,'2283':'Entidades Empresariais'
    ,'2291':'Entidades Empresariais'
    ,'2305':'Entidades Empresariais'
    ,'2313':'Entidades Empresariais'
    ,'3034':'Entidades sem Fins Lucrativos'
    ,'3069':'Entidades sem Fins Lucrativos'
    ,'3077':'Entidades sem Fins Lucrativos'
    ,'3085':'Entidades sem Fins Lucrativos'
    ,'3107':'Entidades sem Fins Lucrativos'
    ,'3115':'Entidades sem Fins Lucrativos'
    ,'3131':'Entidades sem Fins Lucrativos'
    ,'3204':'Entidades sem Fins Lucrativos'
    ,'3212':'Entidades sem Fins Lucrativos'
    ,'3220':'Entidades sem Fins Lucrativos'
    ,'3239':'Entidades sem Fins Lucrativos'
    ,'3247':'Entidades sem Fins Lucrativos'
    ,'3255':'Entidades sem Fins Lucrativos'
    ,'3263':'Entidades sem Fins Lucrativos'
    ,'3271':'Entidades sem Fins Lucrativos'
    ,'3280':'Entidades sem Fins Lucrativos'
    ,'3298':'Entidades sem Fins Lucrativos'
    ,'3301':'Entidades sem Fins Lucrativos'
    ,'3999':'Entidades sem Fins Lucrativos'
    ,'4014':'Pessoas Físicas'
    ,'4022':'Pessoas Físicas'
    ,'4081':'Pessoas Físicas'
    ,'4090':'Pessoas Físicas'
    ,'4111':'Pessoas Físicas'
    ,'4120':'Pessoas Físicas'
    ,'5010':'Organizações Internacionais'
    ,'5029':'Organizações Internacionais'
    ,'5037':'Organizações Internacionais'
    ,'-1':'Ignorado'
}


dict_nat_jur_detail = {
    '1015':'POD EXEC FE'
    ,'1023':'POD EXEC ES'
    ,'1031':'POD EXEC MU'
    ,'1040':'POD LEG FED'
    ,'1058':'POD LEG EST'
    ,'1066':'POD LEG MUN'
    ,'1074':'POD JUD FED'
    ,'1082':'POD JUD EST'
    ,'1104':'AUTARQ FED'
    ,'1112':'AUTARQ EST'
    ,'1120':'AUTARQ MUN'
    ,'1139':'FUNDAC FED'
    ,'1147':'FUNDAC EST'
    ,'1155':'FUNDAC MUN'
    ,'1163':'ORG AUT FED'
    ,'1171':'ORG AUT EST'
    ,'1180':'ORG AUT MUN'
    ,'1198':'COM POLINAC'
    ,'1201':'FUNDO PUBLIC'
    ,'1210':'ASSOC PUBLIC'
    ,'1228':'CONS PUB D PRIV'
    ,'1236':'UF'
    ,'1244':'MUN'
    ,'1252':'FUN PUB PRIV FED'
    ,'1260':'FUN PUB PRIV UF'
    ,'1279':'FUN PUB PRIV MUN'
    ,'2011':'EMP PUB'
    ,'2038':'SOC MISTA'
    ,'2046':'SA ABERTA'
    ,'2054':'SA FECH'
    ,'2062':'SOC QT LTDA'
    ,'2070':'SOC COLETV'
    ,'2089':'SOC COMD SM'
    ,'2097':'SOC COMD AC'
    ,'2127':'SOC CTA PAR'
    ,'2135':'FRM MER IND'
    ,'2143':'COOPERATIVA'
    ,'2151':'CONS EMPRES'
    ,'2160':'GRUP SOC'
    ,'2178':'FIL EMP EXT'
    ,'2194':'FIL ARG-BRA'
    ,'2216':'EMP DOM EXT'
    ,'2224':'FUN INVEST'
    ,'2232':'SOC SIMP PUR'
    ,'2240':'SOC SIMP LTD'
    ,'2259':'SOC SIMP COL'
    ,'2267':'SOC SIMP COM'
    ,'2275':'EMPR BINAC'
    ,'2283':'CONS EMPREG'
    ,'2291':'CONS SIMPLES'
    ,'2305':'EMP IND LTDA'
    ,'2313':'EMP IND SIMPLES'
    ,'3034':'CARTORIO'
    ,'3069':'OUT FUND PR'
    ,'3077':'SERV SOC AU'
    ,'3085':'CONDOMIN'
    ,'3107':'COM CONC'
    ,'3115':'ENT MED ARB'
    ,'3131':'ENT SOCIAL07'
    ,'3204':'FIL FUN EXT'
    ,'3212':'FUN DOM EXT'
    ,'3220':'ORG RELIG'
    ,'3239':'COMUN INDIG'
    ,'3247':'FUNDO PRIVAD'
    ,'3255':'PARTIDO'
    ,'3263':'PARTIDO'
    ,'3271':'PARTIDO'
    ,'3280':'PARTIDO'
    ,'3298':'FRENTE PLEB'
    ,'3301':'OS'
    ,'3999':'OUTR ORG'
    ,'4014':'EMP IND IMO'
    ,'4022':'SEG ESPEC'
    ,'4081':'CONTR IND07'
    ,'4090':'CAN CARG POL'
    ,'4111':'LEILOEIRO'
    ,'4120':'PROD RURAL'
    ,'5010':'ORG INTERNAC'
    ,'5029':'REPR DIPL ES'
    ,'5037':'OUT INST EXT'
    ,'-1':'INGNORADO'  
}


dict_escolaridade = {
    -1:'Não-Superior'
    ,1:'Não-Superior'
    ,2:'Não-Superior'
    ,3:'Não-Superior'
    ,4:'Não-Superior'
    ,5:'Não-Superior'
    ,6:'Não-Superior'
    ,7:'Não-Superior'
    ,8:'Não-Superior'
    ,9:'Superior'
    ,10:'Mestrado'
    ,11:'Doutorado'
}

dict_escolaridade1 = {
    -1:'Inferior ao Ensino Médio Completo'
    ,1:'Inferior ao Ensino Médio Completo'
    ,2:'Inferior ao Ensino Médio Completo'
    ,3:'Inferior ao Ensino Médio Completo'
    ,4:'Inferior ao Ensino Médio Completo'
    ,5:'Inferior ao Ensino Médio Completo'
    ,6:'Inferior ao Ensino Médio Completo'
    ,7:'Ensino Médio ao Superior Incompleto'
    ,8:'Ensino Médio ao Superior Incompleto'
    ,9:'Superior'
    ,10:'Mestrado'
    ,11:'Doutorado'
}